# 準備

In [1]:
import getpass
import os

import gradio as gr

# OpenAI API キーの設定
api_key = getpass.getpass("OpenAI API キーを入力してください: ")
os.environ["OPENAI_API_KEY"] = api_key

# Gradio の基礎

## 簡単なインターフェースの実装

In [2]:
def text2text(text):
    text = "<<" + text + ">>"
    return text

In [ ]:
input_text = gr.Text(label="入力")
output_text = gr.Text(label="出力")

demo = gr.Interface(inputs=input_text, outputs=output_text, fn=text2text)
demo.launch(debug=True)

## ブロックの実装

In [4]:
def text2text_rich(text):
    top = "^" * len(text)
    bottom = "v" * len(text)
    text = f" {top}\n<{text}>\n {bottom}"
    return text

In [ ]:
with gr.Blocks() as demo:
    input_text = gr.Text(label="入力")
    button1 = gr.Button(value="Normal")
    button2 = gr.Button(value="Rich")
    output_text = gr.Text(label="出力")

    button1.click(inputs=input_text, outputs=output_text, fn=text2text)
    button2.click(inputs=input_text, outputs=output_text, fn=text2text_rich)
demo.launch()

## 重要なコンポーネント

In [ ]:
def audio_upload(audio):
    return audio


with gr.Blocks() as demo:
    # Audio
    audio = gr.Audio(label="音声", type="filepath")
    # Checkbox
    checkbox = gr.Checkbox(label="チェックボックス")
    # File
    file = gr.File(label="ファイル", file_types=["image"])
    # Number
    number = gr.Number(label="数値")
    # Markdown
    markdown = gr.Markdown(label="Markdown", value="# タイトル\n## サブタイトル\n本文")
    # Slider
    slider = gr.Slider(
        label="スライダー", minimum=-10, maximum=10, step=0.5, interactive=True
    )
    # Textbox
    textbox = gr.Textbox(label="テキストボックス")

demo.launch(height=1200)

## UI の工夫

In [ ]:
with gr.Blocks() as demo:
    # Accordion
    with gr.Accordion(label="アコーディオン"):
        gr.Text(value="アコーディオンの中身")
    with gr.Row():
        gr.Text(value="左")
        gr.Text(value="右")

    with gr.Row():
        with gr.Column():
            gr.Text(value="(0, 0)")
            gr.Text(value="(1, 0)")
        with gr.Column():
            gr.Text(value="(0, 1)")
            gr.Text(value="(1, 1)")

    with gr.Tab(label="タブ1"):
        gr.Text(value="コンテンツ1")
    with gr.Tab(label="タブ2"):
        gr.Text(value="コンテンツ2")

demo.launch(height=800)

In [ ]:
with gr.Blocks() as demo:
    slider = gr.Slider(label="個数", minimum=0, maximum=10, step=1)

    @gr.render(inputs=slider)
    def render_blocks(value):
        for i in range(value):
            gr.Text(value=f"Block {i}")


demo.launch()

In [ ]:
import time


def iterative_output():
    for i in range(10):
        time.sleep(0.5)
        yield str(i)


with gr.Blocks() as demo:
    button = gr.Button("実行")
    output = gr.Text(label="出力")
    button.click(outputs=output, fn=iterative_output)

demo.launch()

## 状態を保持する

In [ ]:
with gr.Blocks() as demo:
    username = gr.State("")
    text_input = gr.Text(label="ユーザ名")
    button1 = gr.Button("決定")
    button2 = gr.Button("自分の名前を表示")
    text_output = gr.Text(label="出力")
    button1.click(inputs=text_input, outputs=username, fn=lambda x: x)
    button2.click(inputs=username, outputs=text_output, fn=lambda x: x)

demo.launch()

## チャット UI を作る

In [ ]:
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")


def history2messages(history):
    messages = []
    for user, assistant in history:
        messages.append({"role": "user", "content": user})
        messages.append({"role": "assistant", "content": assistant})
    return messages


def chat(message, history):
    messages = history2messages(history)
    messages.append({"role": "user", "content": message})
    response = llm.invoke(message)
    return response.content


demo = gr.ChatInterface(chat)

demo.launch()

## Stream チャットボット

In [ ]:
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")


def chat(message, history):
    messages = history2messages(history)
    messages.append({"role": "user", "content": message})
    output = ""
    for chunk in llm.stream(messages):
        output += chunk.content
        yield output


demo = gr.ChatInterface(chat)

demo.launch(debug=True)

# Gradio の応用

## 翻訳アプリケーション

In [13]:
# 2.2.3 の Runnable
from langchain_core.prompts import PromptTemplate
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

# 1 テンプレートの作成
TRANSLATION_PROMPT = """\
以下の文章を {language} に翻訳し、翻訳結果のみを返してください。
{source_text}
"""
prompt = PromptTemplate.from_template(TRANSLATION_PROMPT)

# 2 Runnable の作成
runnable = prompt | llm

In [ ]:
languages = ["日本語", "英語", "中国語", "ラテン語", "ギリシャ語"]


def translate(source_text, language):
    # 3 Runnable の実行
    response = runnable.invoke(dict(source_text=source_text, language=language))
    return response.content


with gr.Blocks() as demo:
    # 入力テキスト
    source_text = gr.Textbox(label="翻訳元の文章")
    # 言語を選択
    language = gr.Dropdown(label="言語", choices=languages)
    button = gr.Button("翻訳")
    # 出力テキスト
    translated_text = gr.Textbox(label="翻訳結果")

    button.click(inputs=[source_text, language], outputs=translated_text, fn=translate)

demo.launch()

## テーブル作成アプリケーション

In [70]:
# 2.2.4 の Runnable
import csv

import pandas as pd
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import tool
from langchain_openai.chat_models import ChatOpenAI


# 1 入力形式の定義
class CSV2DFToolInput(BaseModel):
    csv_text: str = Field(description="CSVのテキスト")


# 2 ツール本体の定義. csv を保存するツールから json に変換するツールに変更
@tool("csv2json-tool", args_schema=CSV2DFToolInput, return_direct=True)
def csv2json(csv_text: str) -> pd.DataFrame:
    """CSV テキストを pandas DataFrame に変換する"""
    try:
        rows = list(csv.reader(csv_text.splitlines()))
        df = pd.DataFrame(rows[1:], columns=rows[0])
    except Exception:
        df = pd.DataFrame()
    return df.to_json()


# 3 ツールを LLM に紐づける
llm = ChatOpenAI(model="gpt-4o-mini")
# bind するツールを変更
tools = [csv2json]
llm_with_tool = llm.bind_tools(tools=tools, tool_choice="csv2json-tool")

# プロンプトを修正
TABLE_PROMPT = """\
{user_input}
結果は CSV で作成し、csv2json-tool を利用して json に変換してください。
"""
prompt = PromptTemplate.from_template(TABLE_PROMPT)


# 4 Runnable の作成
def get_tool_args(x):
    return x.tool_calls[0]  # AIMessage から ToolCall オブジェクトを取り出す。


runnable = prompt | llm_with_tool | get_tool_args | csv2json

In [ ]:
def create_df(user_input):
    response = runnable.invoke(dict(user_input=user_input))
    json_str = response.content
    df = pd.read_json(json_str)
    return df


with gr.Blocks() as demo:
    # 入力テキスト
    user_input = gr.Textbox(label="テーブルを作成したい内容のテキスト")
    button = gr.Button("実行")
    # 出力テキスト
    output_table = gr.DataFrame()

    button.click(inputs=user_input, outputs=output_table, fn=create_df)

demo.launch(height=1000)

## Plan-and-Solve チャットボット

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [9]:
from langchain_core.pydantic_v1 import BaseModel, Field


# ツール入力形式の定義
class ActionItem(BaseModel):
    action_name: str = Field(description="アクション名")
    action_description: str = Field(description="アクションの詳細")


class Plan(BaseModel):
    """アクションプランを格納する"""

    problem: str = Field(description="問題の説明")
    actions: list[ActionItem] = Field(description="実行すべきアクションリスト")


class ActionResult(BaseModel):
    """実行時の考えと結果を格納する"""

    thoughts: str = Field(description="検討内容")
    result: str = Field(description="結果")

In [10]:
# 単一アクションの実行
from langchain_openai.output_parsers.tools import PydanticToolsParser
from langchain_core.prompts import PromptTemplate


ACTION_PROMPT = """\
問題をアクションプランに分解して解いています。
これまでのアクションの結果と、次に行うべきアクションを示すので、実際にアクションを実行してその結果を報告してください。
# 問題
{problem}
# アクションプラン
{action_items}
# これまでのアクションの結果
{action_results}
# 次のアクション
{next_action}
"""

llm_action = llm.bind_tools([ActionResult], tool_choice="ActionResult")
action_parser = PydanticToolsParser(tools=[ActionResult], first_tool_only=True)
plan_parser = PydanticToolsParser(tools=[Plan], first_tool_only=True)

action_prompt = PromptTemplate.from_template(ACTION_PROMPT)
action_runnable = action_prompt | llm_action | action_parser

In [11]:
# プランに含まれるアクションを実行するループ
def action_loop(action_plan: Plan):
    problem = action_plan.problem
    actions = action_plan.actions

    action_items = "\n".join(["* " + action.action_name for action in actions])
    action_results = []
    action_results_str = ""
    for _, action in enumerate(actions):
        next_action = f"* {action.action_name}  \n{action.action_description}"
        response = action_runnable.invoke(
            dict(
                problem=problem,
                action_items=action_items,
                action_results=action_results_str,
                next_action=next_action,
            )
        )
        action_results.append(response)
        action_results_str += f"* {action.action_name}  \n{response.result}\n"
        yield (
            response.thoughts,
            response.result,
        )  # 変更ポイント: 途中結果を yield で返す

In [12]:
# 全体を通した Runnable 作成
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage

PLAN_AND_SOLVE_PROMPT = """\
ユーザの質問が複雑な場合は、アクションプランを作成し、その後に1つずつ実行する Plan-and-Solve 形式をとります。
これが必要と判断した場合は、Plan ツールによってアクションプランを保存してください。
"""
system_prompt = SystemMessage(PLAN_AND_SOLVE_PROMPT)
chat_prompt = ChatPromptTemplate.from_messages(
    [system_prompt, MessagesPlaceholder(variable_name="history")]
)

llm_plan = llm.bind_tools(tools=[Plan])
planning_runnable = chat_prompt | llm_plan  # route を削除

In [24]:
from gradio import ChatMessage
from langchain_core.messages import AIMessage, HumanMessage


def chat(prompt, messages, history):
    # 描画用の履歴をアップデート
    messages.append(ChatMessage(role="user", content=prompt))
    # LangChain 用の履歴をアップデート
    history.append(HumanMessage(content=prompt))
    # プランまたは返答を作成
    response = planning_runnable.invoke(dict(history=history))
    if response.response_metadata["finish_reason"] != "tool_calls":
        # タスクが簡単な場合はプランを作らずに返す
        messages.append(ChatMessage(role="assistant", content=response.content))
        history.append(AIMessage(content=response.content))
        yield "", messages, history
    else:
        # アクションプランを抽出
        action_plan = plan_parser.invoke(response)

        # アクション名を表示
        action_items = "\n".join(
            ["* " + action.action_name for action in action_plan.actions]
        )
        messages.append(
            ChatMessage(
                role="assistant",
                content=action_items,
                metadata={"title": "実行されるアクション"},
            )
        )
        # プランの段階で一度描画する
        yield "", messages, history

        # アクションプランを実行
        action_results_str = ""
        for i, (thoughts, result) in enumerate(action_loop(action_plan)):
            action_name = action_plan.actions[i].action_name
            action_results_str += f"* {action_name}  \n{result}\n"
            text = f"## {action_name}\n### 思考過程\n{thoughts}\n### 結果\n{result}"
            messages.append(ChatMessage(role="assistant", content=text))
            # 実行結果を描画する
            yield "", messages, history

        history.append(AIMessage(content=action_results_str))
        # LangChain 用の履歴を更新する
        yield "", messages, history

In [ ]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Assistant", type="messages", height=800)
    history = gr.State([])
    with gr.Row():
        with gr.Column(scale=9):
            user_input = gr.Textbox(lines=1, label="Chat Message")
        with gr.Column(scale=1):
            submit = gr.Button("Submit")
            clear = gr.ClearButton([user_input, chatbot, history])
    submit.click(
        chat,
        inputs=[user_input, chatbot, history],
        outputs=[user_input, chatbot, history],
    )
demo.launch(height=1000)